In [38]:
import numpy as np
import pandas as pd
import json
import os

In [39]:
def read_json_files(directory):
    files = os.listdir(directory)
    json_files = [file for file in files if file.endswith('.json')]

    dfs = []
    for file in json_files:
        file_path = os.path.join(directory, file)
        json_data = pd.read_json(file_path)
        dfs.append(json_data)

    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

df = read_json_files('.')

df

,image,objects
0,"{'fileName': '20220622_SD04_7973.JPG', 'detail...","[{'bndbox': {'minX': 0.904863, 'minY': 0.90394..."
1,"{'fileName': '20220622_SD04_7892.JPG', 'detail...","[{'bndbox': {'minX': 0.560254, 'minY': 0.41549..."
2,"{'fileName': '20220622_SD04_7779.JPG', 'detail...","[{'bndbox': {'minX': 0.48625799999999997, 'min..."
3,"{'fileName': '20220622_SD04_7965.JPG', 'detail...","[{'bndbox': {'minX': 0.293869, 'minY': 0.54554..."
4,"{'fileName': '20220622_SD04_7957.JPG', 'detail...","[{'bndbox': {'minX': 0.20507399999999998, 'min..."
...,...,...
6746,"{'fileName': '20220627_SD04_8847.JPG', 'detail...","[{'bndbox': {'minX': 0.21301799999999999, 'min..."
6747,"{'fileName': '20220627_SD04_8804.JPG', 'detail...","[{'bndbox': {'minX': 0.5897439999999999, 'minY..."
6748,"{'fileName': '20220627_SD04_8595.JPG', 'detail...","[{'bndbox': {'minX': 0.545652, 'minY': 0.20220..."
6749,"{'fileName': '20220627_SD04_8768.JPG', 'detail...","[{'bndbox': {'minX': 0.268245, 'minY': 0.04438..."


In [40]:
image = df['image'].to_dict()
file_n = []
for i in range(0, len(image)):
    file_n.append(image[i]['fileName'])
    
file_orientation =[]
for i in range(0, len(image)):
    if image[i]['details']['width'] < image[i]['details']['height']:
        orientation_s = 'portait'
        file_orientation.append(orientation_s)
    else: 
        orientation_s = 'landscape'
        file_orientation.append(orientation_s)

jpg_df = pd.DataFrame({'file_name': file_n, 'orientation': file_orientation})
jpg_df = jpg_df.sort_values('file_name')
ind = jpg_df.index
jpg_df = jpg_df.reset_index(drop=True)
jpg_df = jpg_df.reset_index().rename(columns={'index': 'jpg_id'})
jpg_df

,jpg_id,file_name,orientation
0,0,20220308_SD02_2923.JPG,portait
1,1,20220308_SD02_2924.JPG,portait
2,2,20220308_SD02_2946.JPG,portait
3,3,20220308_SD02_2954.JPG,landscape
4,4,20220308_SD02_2955.JPG,landscape
...,...,...,...
6746,6746,20220825_SD05_6957.JPG,portait
6747,6747,20220825_SD05_6958.JPG,portait
6748,6748,20220825_SD05_6960.JPG,portait
6749,6749,20220825_SD05_6963.JPG,portait


In [41]:
ind

Int64Index([5801, 5800, 2462, 2463, 2450, 2456, 2479, 2471, 2473, 2476,
            ...
             109,  116,  128,  112,  120,  104,  108,  117,  127,  114],
           dtype='int64', length=6751)

In [42]:
obj = df['objects'].to_dict()

In [43]:
coord_list = []
bndbox_name_list = []
for i in range(len(obj)):
    coord = []
    bndbox_name = []
    for j in range(len(obj[i])):
        category_name = obj[i][j]['category']['name']
        bndbox_dict = obj[i][j]['bndbox']
        bndbox_tuple = (bndbox_dict['minX'], bndbox_dict['minY'], bndbox_dict['maxX'], bndbox_dict['maxY'])
        
        coord.append(bndbox_tuple)
        bndbox_name.append(category_name)
    coord_list.append(coord)
    bndbox_name_list.append(bndbox_name)

bndbox_df = pd.DataFrame({'bndbox_name': bndbox_name_list, 'coordinates': coord_list})
bndbox_df = bndbox_df.reindex(ind)
bndbox_df = bndbox_df.reset_index(drop=True)
bndbox_df = bndbox_df.reset_index().rename(columns={'index': 'jpg_id'})
bndbox_df.head()

,jpg_id,bndbox_name,coordinates
0,0,"[DF2I_Face, DF2I_Body]","[(0.36186799999999997, 0.0028209999999999997, ..."
1,1,"[DF2I_Face, DF2I_Body]","[(0.34282199999999996, 0.0, 0.8845649999999999..."
2,2,"[151J_Face, 151J_Body, 110E_Face, 110E_Body, 7...","[(0.152365, 0.35543, 0.5100009999999999, 0.558..."
3,3,"[151J_Face, 151J_Body, 110E_Face, 110E_Body, 7...","[(0.42407799999999995, 0.587402, 0.61605199999..."
4,4,"[151J_Face, 151J_Body, 110E_Face, 110E_Body, 7...","[(0.47397000000000006, 0.564622, 0.681128, 0.8..."


In [44]:
bndbox_df['coordinates'][0]

[(0.36186799999999997, 0.0028209999999999997, 0.916308, 0.38222799999999996),
 (0.0, 0.0, 0.954399, 1.0)]

In [45]:
bndbox_df = bndbox_df.explode(['bndbox_name', 'coordinates'])
bndbox_df = bndbox_df.sort_values('jpg_id')
bndbox_df.head(15)

,jpg_id,bndbox_name,coordinates
0,0,DF2I_Face,"(0.36186799999999997, 0.0028209999999999997, 0..."
0,0,DF2I_Body,"(0.0, 0.0, 0.954399, 1.0)"
1,1,DF2I_Face,"(0.34282199999999996, 0.0, 0.8845649999999999,..."
1,1,DF2I_Body,"(0.0, 0.0028209999999999997, 0.975560999999999..."
2,2,7124_Body,"(0.002116, 0.0, 0.387262, 0.7334269999999999)"
2,2,110E_Body,"(0.101577, 0.09591000000000001, 0.971329, 1.0)"
2,2,7124_Face,"(0.0, 0.0, 0.378797, 0.31029599999999996)"
2,2,151J_Body,"(0.126971, 0.35543, 0.8591709999999999, 0.943583)"
2,2,151J_Face,"(0.152365, 0.35543, 0.5100009999999999, 0.558533)"
2,2,110E_Face,"(0.118506, 0.094499, 0.5100009999999999, 0.389..."


In [57]:
toy_coord = bndbox_df[bndbox_df['bndbox_name'].str.contains('toy', case=False)]
monkey_coord = bndbox_df[~bndbox_df['bndbox_name'].str.contains('toy', case=False)]

# Toy Coord

In [58]:
toy_coord = toy_coord[~toy_coord['bndbox_name'].str.contains('triangle|swing', case=False)]
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace(r'Toy-|Toy_|toy_|toy-', '', regex=True)
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace('Blue Frisbees', 'Blue Frisbee')
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace('Lucky Clover chew', 'Lucky Clover Chew')
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace(' ', '')

toy_coord.bndbox_name.unique()

array(['Unknown', 'DentalStar', 'Dumbbell', 'Grenade', 'Kong',
       'LuckyCloverChew', 'MegalastBall', 'Ziggs', 'BlueFrisbee',
       'DentalKong', 'Zyro', 'Hex', 'S-shapedToy', 'ChallengerBall'],
      dtype=object)

In [59]:
toy_coord = toy_coord.rename(columns={'bndbox_name': 'toy'})
toy_coord

,jpg_id,toy,coordinates
34,34,Unknown,"(0.0, 0.895628, 0.16083, 1.0)"
38,38,Unknown,"(0.7477809999999999, 0.818182, 1.0, 0.97861000..."
60,60,DentalStar,"(0.6565219999999999, 0.727354, 0.947826, 0.825..."
80,80,Dumbbell,"(0.321212, 0.748602, 1.0, 1.0)"
163,163,Grenade,"(0.40405599999999997, 0.365112, 0.528861, 0.61..."
...,...,...,...
6653,6653,ChallengerBall,"(0.5826089999999999, 0.0, 1.0, 0.3202100000000..."
6654,6654,ChallengerBall,"(0.6869569999999999, 0.001449, 1.0, 0.22458199..."
6695,6695,Kong,"(0.8576849999999999, 0.8910309999999999, 0.944..."
6696,6696,Kong,"(0.8842500000000001, 0.928039, 0.963947, 1.0)"


# Monkey Coord

In [72]:
monkey_coord = monkey_coord[~monkey_coord['bndbox_name'].str.contains('triangle|swing', case=False)]
# Define the dictionary of replacements
replacements = {'Unknown 1': 'Unknown1', 'Unknown 2': 'Unknown2', 'DF21' : 'DF2I', 'OEX': '0EX', '0F1': '0FL'}
# Replace the patterns in the 'bndbox_name' column using the dictionary of replacements
monkey_coord['bndbox_name'] = monkey_coord['bndbox_name'].replace(replacements, regex=True)

monkey_coord['bndbox_name'] = monkey_coord['bndbox_name'].str.replace('Unknown ', 'Unknown1 ')
monkey_coord = monkey_coord[~monkey_coord['bndbox_name'].str.contains('Unknown3')]

In [73]:
monkey_coord.bndbox_name.unique()

array(['DF2I_Face', 'DF2I_Body', '7124_Body', '110E_Body', '7124_Face',
       '151J_Body', '151J_Face', '110E_Face', '67G_Face', '67G_Body',
       '94B_Body', '94B_Face', '72X_Face', '72X_Body', '143H_Body',
       '143H_Face', '69X_Face', '69X_Body', '87J_Body', '87J_Face',
       '14F_Face', '14F_Body', '19J_Face', '19J_Body', '101G_Face',
       'G701_Face', 'G701_Body', '68F_Body', '68F_Face', 'Unknown1 Body',
       '151J Body', '151J Face', '110E Body', '110E Face', '7124 Face',
       '7124 Body', '49Y Face', '49Y Body', 'Unknown1 Face', '79G Face',
       '79G Body', '68E Face', '68E Body', '167I Face', '167I Body',
       '86I Face', '86I Body', '59E Face', '59E Body', '134J Body',
       '134J  Face', 'Unknown2 Body', 'Unknown2 Face', '35Y Body',
       '35Y Face', 'G942 Face', 'G942 Body', '144H Body', '144H Face',
       '58I Face', '58I Body', '46J Body', '46J Face', '26J Body',
       '26J Face', 'DF2I Face', 'DF2I Body', '48Z Face', '48Z Body',
       '70G Body', '70G 

In [71]:
monkey_coord = monkey_coord.reset_index(drop=True)
monkey_coord.head()

,jpg_id,bndbox_name,coordinates
0,0,DF2I_Face,"(0.36186799999999997, 0.0028209999999999997, 0..."
1,0,DF2I_Body,"(0.0, 0.0, 0.954399, 1.0)"
2,1,DF2I_Face,"(0.34282199999999996, 0.0, 0.8845649999999999,..."
3,1,DF2I_Body,"(0.0, 0.0028209999999999997, 0.975560999999999..."
4,2,7124_Body,"(0.002116, 0.0, 0.387262, 0.7334269999999999)"


In [75]:
monkey_coord[monkey_coord['bndbox_name'].str.contains('Unknown', case=False)]

,jpg_id,bndbox_name,coordinates
124,46,Unknown1 Body,"(0.728653, 0.0, 1.0, 1.0)"
139,50,Unknown1 Face,"(0.2, 0.24052, 0.267391, 0.340495)"
140,50,Unknown1 Body,"(0.184783, 0.250662, 0.8326089999999999, 0.980..."
170,60,Unknown2 Body,"(0.47801299999999997, 0.0, 0.606784, 0.1209359..."
171,60,Unknown1 Face,"(0.291304, 0.308618, 0.526087, 0.4506119999999..."
...,...,...,...
21018,6740,Unknown1 Body,"(0.324692, 0.42715699999999995, 0.995297999999..."
21023,6741,Unknown1 Body,"(0.288615, 0.38048099999999996, 0.988931999999..."
21026,6741,Unknown1 Face,"(0.30347, 0.6605369999999999, 0.643018, 0.817539)"
21029,6742,Unknown1 Face,"(0.24192799999999998, 0.649222, 0.592086, 0.82..."


In [91]:
max_jpgid = monkey_coord['jpg_id'].max()
min_jpgid = monkey_coord['jpg_id'].min()

count = 0
for i in range(min_jpgid, max_jpgid+1):
    temp = monkey_coord[monkey_coord['jpg_id']==i]
    if temp['bndbox_name'].duplicated().any():
        print(temp)
        count += 1
        
print(count)
    
    

      jpg_id    bndbox_name                                        coordinates
4897    1657       67K Body  (0.37826099999999996, 0.423979, 0.491303999999...
4898    1657       0EX Face            (0.369565, 0.140239, 0.534783, 0.45007)
4899    1657       0EX Body  (0.34782599999999997, 0.127194, 0.797825999999...
4900    1657  Unknown1 Body          (0.0, 0.0, 0.265217, 0.30983099999999997)
4901    1657  Unknown1 Body                (0.0, 0.381581, 0.115217, 0.541389)
      jpg_id bndbox_name                                        coordinates
5034    1684    09X Body     (0.0, 0.519786, 0.22305000000000003, 0.917647)
5035    1684    09X Body  (0.12997899999999998, 0.530481, 0.494241, 0.97...
5036    1684    09X Face  (0.131584, 0.53369, 0.32254099999999997, 0.687...
      jpg_id    bndbox_name                                        coordinates
5180    1719       26J Body  (0.17307699999999998, 0.030761999999999998, 0....
5181    1719       26J Face           (0.319231, 0.026916, 0.576

       jpg_id bndbox_name                                        coordinates
12482    4026   167I Body                 (0.245436, 0.36502, 0.945233, 1.0)
12483    4026   167I Face           (0.545639, 0.366372, 0.858012, 0.569161)
12484    4026    59E Body                (0.0, 0.164935, 0.701826, 0.978795)
12485    4026    59E Body                     (0.602434, 0.132489, 1.0, 1.0)
12486    4026    59E Face  (0.11967499999999999, 0.16763899999999998, 0.4...
12487    4026    59E Face  (0.616633, 0.136545, 0.9675459999999999, 0.333...
       jpg_id    bndbox_name  \
12655    4073  Unknown1 Body   
12656    4073      114J Face   
12657    4073      114J Body   
12658    4073  Unknown1 Body   

                                             coordinates  
12655                       (0.6714, 0.846306, 1.0, 1.0)  
12656  (0.5212979999999999, 0.03515, 0.770791, 0.278497)  
12657  (0.25557799999999997, 0.033798, 0.88641, 0.851...  
12658                 (0.324544, 0.77195, 0.653144, 1.0)  
     

In [67]:
jpg_df[jpg_df.jpg_id==245]

,jpg_id,file_name,orientation
245,245,20220331_SD02_3494.JPG,portait


In [68]:
monkey_coord[monkey_coord.jpg_id==245]

,jpg_id,bndbox_name,coordinates
245,245,Unknown1 Face,"(0.040117, 0.32192499999999996, 0.282424, 0.49..."
245,245,Unknown1 Body,"(0.008022999999999999, 0.17754, 0.657918999999..."
245,245,Unknown2 Face,"(0.046536, 0.7914439999999999, 0.410798, 0.982..."
245,245,Unknown2 Body,"(0.044931, 0.618182, 0.9178769999999999, 1.0)"
245,245,Unknown3 Body,"(0.0, 0.10287299999999999, 0.43913, 0.86065399..."


In [66]:
monkey_coord[monkey_coord['bndbox_name'].str.contains('Unknown3 ')]

,jpg_id,bndbox_name,coordinates
245,245,Unknown3 Body,"(0.0, 0.10287299999999999, 0.43913, 0.86065399..."


In [32]:
monkey_coord[['monkey_name', 'body_part']] = monkey_coord['bndbox_name'].str.split('_| ', 1, expand=True)
monkey_coord = monkey_coord.drop('bndbox_name',axis=1)

In [33]:
monkey_coord['monkey_name'] = monkey_coord['monkey_name'].str.replace(' ', '')
monkey_coord.monkey_name.unique()

array(['DF2I', '7124', '110E', '151J', '67G', '94B', '72X', '143H', '69X',
       '87J', '14F', '19J', '101G', 'G701', '68F', 'Unknown1', '49Y',
       '79G', '68E', '167I', '86I', '59E', '134J', 'Unknown2', '35Y',
       'G942', '144H', '58I', '46J', '26J', '48Z', '70G', '114J', '42Z',
       '36J', '0EX', '0FL', '42K', 'Unknown3', '37I', '68Y', '81G', '40J',
       '09X', '58K', '59K', '67K', '129K', '98K', '120K', '126K', '139K',
       '122K', 'Unknown', '170K'], dtype=object)

In [34]:
monkey_coord['body_part'] = monkey_coord['body_part'].str.replace(' ', '')
monkey_coord['body_part'] = monkey_coord['body_part'].str.capitalize()
monkey_coord.body_part.unique()

array(['Face', 'Body'], dtype=object)

In [35]:
monkey_coord.tail(5)

,jpg_id,coordinates,monkey_name,body_part
6748,6748,"(0.0, 0.173975, 0.8955559999999999, 0.876945)",G942,Body
6749,6749,"(0.581475, 0.164074, 0.846746, 0.424328)",G942,Face
6749,6749,"(0.0021219999999999998, 0.168317, 1.0, 0.929279)",G942,Body
6750,6750,"(0.22919399999999998, 0.011314999999999999, 0....",G942,Face
6750,6750,"(0.0042439999999999995, 0.015559, 0.965588, 1.0)",G942,Body


In [36]:
jpg_df.to_csv('../fromjson_photos.csv', index=False)
monkey_coord.to_csv('../fromjson_mCoord.csv', index=False)
toy_coord.to_csv('../fromjson_toyCoord.csv', index = False)